<a href="https://colab.research.google.com/github/alex-jk/painting-lora-finetune/blob/main/paintings_lora_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip -O paintings.zip
!unzip -o paintings.zip -d paintings/

--2025-04-08 00:26:44--  https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7164316 (6.8M) [application/zip]
Saving to: ‘paintings.zip’

paintings.zip       100%[===================>]   6.83M  --.-KB/s    in 0.1s    

2025-04-08 00:26:45 (70.7 MB/s) - ‘paintings.zip’ saved [7164316/7164316]

Archive:  paintings.zip
   creating: paintings/paintings/
  inflating: paintings/paintings/17030-islands-on-kabenung-40x60-brigitte-granton_orig.jpeg  
  inflating: paintings/paintings/23060-Setting-in-a-Grand-Way-Grand-Lake-30X60_-3240.jpeg  
  inflating: paintings/paintings/24055-Sunswept-Opeongo-40x48_-oil-Algonquin-G-539x443.jpeg  
  inflating: paintings/paintings/30yl2012ivanhoethe

In [2]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers.git

Found existing installation: diffusers 0.32.2
Uninstalling diffusers-0.32.2:
  Successfully uninstalled diffusers-0.32.2
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-p09vfzfo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-p09vfzfo
  Resolved https://github.com/huggingface/diffusers.git to commit 5ded26cdc70387e7d08144275c138dc0c175a4a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-py3-none-any.whl size=3544305 sha256=aa924d01faeb29eeaa7f3bc930c52c97c7b056a10824ac4de7ff840400bb8d73
  Stored in directory: /tmp/pip-ephem-wheel-cache-77f9h3hr/wheels/d2/5c/5f/16639722ea17ecb73ab461b81718584bac08af2801619786b9
Successfully built diffusers


In [21]:
import diffusers
print(diffusers.__version__)

from diffusers import StableDiffusionPipeline, UNet2DConditionModel
import torch

from peft import LoraConfig # TaskType is no longer needed here
import torch.optim as optim

from torchvision import transforms # Import torchvision for image transforms
from torch.utils.data import Dataset, DataLoader # Import Dataset and DataLoader
from PIL import Image # Import PIL for image loading
import os # Import os for path manipulation
import glob
import subprocess # Better way to run shell commands

import numpy as np

0.33.0.dev0


In [6]:
# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [7]:
# Enable LoRA training in the U-Net
pipe.unet.enable_gradient_checkpointing()
# pipe.enable_xformers_memory_efficient_attention()

In [8]:
# Get a reference to the UNet model
unet = pipe.unet
print("UNet reference obtained.")
# 2. Define the LoRA configuration object
#    Omitting task_type as it's not required for unet.add_adapter()
#    and the previous enum value was incorrect.
lora_config = LoraConfig(
    r=4,
    lora_alpha=4,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.05,
    bias="none",
    # task_type=... # REMOVED - Not needed here and caused AttributeError
)

print("LoRA Config defined (without task_type).")

UNet reference obtained.
LoRA Config defined (without task_type).


In [9]:
# 3. Add the LoRA adapter to the UNet model using the defined config
#    The 'adapter_name' is optional but good practice if you plan
#    to manage multiple adapters later.
adapter_name = "my_style_lora" # You can choose any descriptive name
unet.add_adapter(lora_config, adapter_name=adapter_name)

print(f"LoRA adapter '{adapter_name}' added to the UNet.")

# Optional but recommended: Verify trainable parameters
# This helps confirm that only the LoRA weights are set to be trained.
trainable_params = 0
all_param = 0
for name, param in unet.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        # Print trainable parameter names (optional, for debugging)
        # print(f"Trainable: {name}")
        trainable_params += param.numel()

print(
    f"\nTotal UNet params: {all_param:,}\n"
    f"Trainable params (LoRA): {trainable_params:,}\n"
    f"Trainable %: {100 * trainable_params / all_param:.4f}%"
)

LoRA adapter 'my_style_lora' added to the UNet.

Total UNet params: 860,318,148
Trainable params (LoRA): 797,184
Trainable %: 0.0927%


In [10]:
# 4. Set up the Optimizer
#    Filter the UNet's parameters to get only those that require gradients (the LoRA parameters).
lora_params_to_optimize = [p for p in unet.parameters() if p.requires_grad]
print(f"Number of parameter groups to optimize: {len(lora_params_to_optimize)}")

# Define common hyperparameters for the optimizer
learning_rate = 1e-4 # A common starting LR for LoRA training
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_weight_decay = 1e-2
adam_epsilon = 1e-08

# Create the AdamW optimizer, passing only the LoRA parameters
optimizer = optim.AdamW(
    lora_params_to_optimize,
    lr=learning_rate,
    betas=(adam_beta1, adam_beta2),
    weight_decay=adam_weight_decay,
    eps=adam_epsilon,
)

print(f"Optimizer AdamW created with learning rate {learning_rate}.")

Number of parameter groups to optimize: 256
Optimizer AdamW created with learning rate 0.0001.


In [11]:
# Get references to VAE, Text Encoder, Tokenizer, and Scheduler from the pipeline
vae = pipe.vae
text_encoder = pipe.text_encoder
tokenizer = pipe.tokenizer
scheduler = pipe.scheduler # The noise scheduler

print("VAE, Text Encoder, Tokenizer, and Scheduler extracted from pipeline.")

# Freeze VAE and Text Encoder parameters
# These models should not be trained during LoRA fine-tuning of the UNet
vae.requires_grad_(False)
text_encoder.requires_grad_(False)

print("VAE and Text Encoder parameters frozen (requires_grad=False).")

# Optional: Move VAE and Text Encoder to appropriate dtype and device if not already done
# (The pipe.to("cuda") should have handled this, but explicit check/move can be useful)
# vae.to(unet.device, dtype=torch.float16) # Match UNet's device and potentially dtype
# text_encoder.to(unet.device) # Text encoder usually stays float32

# Verify UNet is still in training mode (add_adapter should preserve this)
if not unet.training:
    unet.train()
    print("Set UNet back to training mode.")
else:
    print("UNet is already in training mode.")

VAE, Text Encoder, Tokenizer, and Scheduler extracted from pipeline.
VAE and Text Encoder parameters frozen (requires_grad=False).
Set UNet back to training mode.


In [12]:
# --- Code just for checking the directory ---

# Define the directory where files were supposed to be unzipped
base_data_dir = "paintings"
possible_subdir = os.path.join(base_data_dir, "paintings") # Common pattern if zip contained a 'paintings' folder

print(f"--- Checking contents of '{base_data_dir}' ---")

# Check if the base directory exists
if not os.path.isdir(base_data_dir):
    print(f"Error: Directory '{base_data_dir}' does not exist.")
else:
    # List immediate contents
    print(f"\nDirect contents of '{base_data_dir}':")
    try:
        for item in os.listdir(base_data_dir):
            item_path = os.path.join(base_data_dir, item)
            item_type = "DIR" if os.path.isdir(item_path) else "FILE"
            print(f"- {item} ({item_type})")
    except Exception as e:
        print(f"Error listing contents: {e}")


    # Look for image files directly inside base_data_dir
    image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.webp')
    images_in_base = []
    for ext in image_extensions:
        images_in_base.extend(glob.glob(os.path.join(base_data_dir, ext)))

    if images_in_base:
        print(f"\nFound {len(images_in_base)} image(s) directly in '{base_data_dir}'. Example: {os.path.basename(images_in_base[0])}")
    else:
        print(f"\nNo image files found directly in '{base_data_dir}'.")

    # Check if the common subdirectory exists and look for images there
    if os.path.isdir(possible_subdir):
        print(f"\nChecking common subdirectory '{possible_subdir}'...")
        images_in_subdir = []
        for ext in image_extensions:
            images_in_subdir.extend(glob.glob(os.path.join(possible_subdir, ext)))

        if images_in_subdir:
            print(f"Found {len(images_in_subdir)} image(s) in subdirectory '{possible_subdir}'. Example: {os.path.basename(images_in_subdir[0])}")
        else:
            print(f"No image files found in subdirectory '{possible_subdir}'.")
    else:
         print(f"\nCommon subdirectory '{possible_subdir}' does not exist.")


print("\n--- End Check ---")

--- Checking contents of 'paintings' ---

Direct contents of 'paintings':
- paintings (DIR)

No image files found directly in 'paintings'.

Checking common subdirectory 'paintings/paintings'...
Found 24 image(s) in subdirectory 'paintings/paintings'. Example: the-evening-hour-jake-vandenbrink.jpg

--- End Check ---


In [13]:
# Download, Extract Data, and Prepare Dataset/DataLoader

# Define data source and target directory
data_url = "https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip"
zip_file = "paintings.zip"
data_dir = "paintings" # Directory where data will be unzipped

# Create target directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# Download the data if zip file doesn't exist
if not os.path.exists(zip_file):
    print(f"Downloading data from {data_url}...")
    try:
        # Using subprocess for better error handling than '!' in some environments
        subprocess.run(["wget", data_url, "-O", zip_file], check=True)
        print("Download complete.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading file: {e}")
        # Handle error appropriately, maybe raise exception
        raise SystemExit(f"Failed to download data: {e}")
    except FileNotFoundError:
        print("Error: 'wget' command not found. Please ensure wget is installed.")
        raise SystemExit("wget not found.")
else:
    print(f"Zip file '{zip_file}' already exists. Skipping download.")

# Unzip the data
print(f"Unzipping {zip_file} into {data_dir}...")
try:
    # Use -o to overwrite existing files without prompting
    subprocess.run(["unzip", "-o", zip_file, "-d", data_dir], check=True, capture_output=True)
    print("Unzipping complete.")
except subprocess.CalledProcessError as e:
    print(f"Error unzipping file: {e}")
    print(f"Stderr: {e.stderr.decode()}")
    raise SystemExit(f"Failed to unzip data: {e}")
except FileNotFoundError:
    print("Error: 'unzip' command not found. Please ensure unzip is installed.")
    raise SystemExit("unzip not found.")

Zip file 'paintings.zip' already exists. Skipping download.
Unzipping paintings.zip into paintings...
Unzipping complete.


In [18]:
# Define Hyperparameters for data preparation
image_resolution = 512 # Match the resolution SD v1.5 was trained on
batch_size = 4       # Adjust based on your GPU memory (start small)

# Define Image Transformations
preprocess = transforms.Compose(
    [
        transforms.Resize(image_resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(image_resolution),
        transforms.ToTensor(), # Convert image to tensor [0, 1]
        transforms.Normalize([0.5], [0.5]), # Normalize to [-1, 1]
    ]
)

# Define Custom Dataset pointing to the 'paintings' directory
class PaintingStyleDataset(Dataset):
    def __init__(self, instance_data_root, tokenizer, size=512):
        self.instance_data_root = instance_data_root
        self.tokenizer = tokenizer # Keep tokenizer reference if needed later
        self.size = size

        self.instance_images_path = []
        valid_extensions = ('.png', '.jpg', '.jpeg', '.webp')
        if not os.path.isdir(instance_data_root):
             raise ValueError(f"Dataset directory '{instance_data_root}' not found after unzipping.")

        for filename in os.listdir(instance_data_root):
            if filename.lower().endswith(valid_extensions):
                self.instance_images_path.append(os.path.join(instance_data_root, filename))

        self.num_instance_images = len(self.instance_images_path)
        self._length = self.num_instance_images
        self.image_transforms = preprocess

        if self.num_instance_images == 0:
             print(f"Warning: No valid image files found in {instance_data_root}. Check the directory structure and file extensions.")
        else:
            print(f"Dataset initialized: Found {self.num_instance_images} images in {instance_data_root}")


    def __len__(self):
        return self._length

    def __getitem__(self, index):
        example = {}
        if index >= self.num_instance_images: # Should not happen with DataLoader but safe check
             raise IndexError("Index out of bounds")

        instance_image_path = self.instance_images_path[index]
        try:
            instance_image = Image.open(instance_image_path)
            if not instance_image.mode == "RGB":
                instance_image = instance_image.convert("RGB")
            example["instance_images"] = self.image_transforms(instance_image)
        except Exception as e:
            print(f"Warning: Could not load image {instance_image_path}. Error: {e}. Skipping index {index}.")
            # Return None to indicate failure, handled in collate_fn
            return None

        # Define the text prompt for training.
        # Since we're training a style, use a consistent prompt with a trigger word.
        # <<< IMPORTANT: Replace 'sks painting style' with your desired trigger phrase >>>
        instance_prompt = "oil painting of Canadian nature"
        example["instance_prompt"] = instance_prompt # Store the raw text prompt

        return example

# Define the Collate Function (handles potential None values from failed image loads)
def simple_collate_fn(examples):
    # Filter out None entries (failed image loads)
    valid_examples = [ex for ex in examples if ex is not None]
    if not valid_examples: # Handle case where entire batch failed
        return None # Or however your training loop handles empty batches

    # Stack images
    pixel_values = torch.stack([example["instance_images"] for example in valid_examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()

    # Collect prompts
    prompts = [example["instance_prompt"] for example in valid_examples]

    return {"pixel_values": pixel_values, "prompts": prompts}

In [15]:
actual_image_dir = "paintings/paintings"

# Instantiate the Dataset using the correct path
print(f"Initializing dataset with image path: '{actual_image_dir}'")
train_dataset = PaintingStyleDataset(
    instance_data_root=actual_image_dir, # Use the correct path
    tokenizer=tokenizer,
    size=image_resolution
)

# Create the DataLoader only if the dataset has images
train_dataloader = None
if len(train_dataset) > 0:
    train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=simple_collate_fn, # Assumes simple_collate_fn is defined
        batch_size=batch_size,
        num_workers=2 # Adjust as needed
    )
    print(f"DataLoader created successfully with {len(train_dataset)} images.")

    # Optional: Verify by loading one batch
    # try:
    #     first_batch = next(iter(train_dataloader))
    #     if first_batch:
    #         print("Successfully loaded first batch.")
    #         print("Image batch shape:", first_batch['pixel_values'].shape)
    #         print("Number of prompts in batch:", len(first_batch['prompts']))
    #     else:
    #          print("Warning: First batch loaded as None/Empty.")
    # except Exception as e:
    #      print(f"Error loading first batch: {e}")

else:
    print("Error: Dataset is still empty even with the corrected path. Check permissions or contents of 'paintings/paintings'.")

Initializing dataset with image path: 'paintings/paintings'
Dataset initialized: Found 25 images in paintings/paintings
DataLoader created successfully with 25 images.


In [35]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,  # float16 is normally fine
    safety_checker=None
).to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [36]:
unet = pipe.unet
vae = pipe.vae
tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder

unet = unet.to(torch.float32)

In [37]:
latents = torch.randn(1, 4, 64, 64).to(unet.dtype).to("cuda")  # random noise as input
timesteps = torch.tensor([10]).to("cuda")  # timestep = 10 for testing
prompt = ["a photo of a painting"]

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding="max_length",
    max_length=tokenizer.model_max_length
).to("cuda")

with torch.no_grad():
    encoder_hidden_states = text_encoder(inputs.input_ids)[0].to(unet.dtype)
    output = unet(latents, timesteps, encoder_hidden_states)

print("UNet output sample stats:")
print("  min:", output.sample.min().item())
print("  max:", output.sample.max().item())
print("  has NaN:", torch.isnan(output.sample).any().item())

UNet output sample stats:
  min: -3.467980146408081
  max: 3.521069288253784
  has NaN: False


In [38]:
from diffusers.models.attention_processor import LoRAAttnProcessor

lora_rank = 4  # or 8 if you want slightly more expressive adapters
cross_attention_dim = unet.config.cross_attention_dim
attn_procs = {}

# Get number of channels per block (used to set correct hidden sizes)
block_out_channels = unet.config.block_out_channels

# Build LoRA processors for each attention layer
for name in unet.attn_processors.keys():
    if "mid_block" in name:
        hidden_size = block_out_channels[-1]
    elif "up_blocks" in name:
        block_id = int(name.split(".")[1])
        hidden_size = list(reversed(block_out_channels))[block_id]
    elif "down_blocks" in name:
        block_id = int(name.split(".")[1])
        hidden_size = block_out_channels[block_id]
    else:
        continue

    attn_procs[name] = LoRAAttnProcessor(
        hidden_size, cross_attention_dim, lora_rank
    )

# Inject into U-Net
unet.set_attn_processor(attn_procs)

TypeError: LoRAAttnProcessor.__init__() takes 1 positional argument but 4 were given

In [34]:
from tqdm.auto import tqdm # For progress bar
import torch.nn.functional as F # For loss function

# --- NEXT STEP ---

# 7. Set up and Run the Training Loop

# Training Hyperparameters
num_train_epochs = 3  # Adjust as needed (start small, maybe 50-200 for LoRA)
gradient_accumulation_steps = 1 # Increase if batch size is small and GPU memory allows
learning_rate = 1e-4 # Should match the optimizer's LR if not using a scheduler explicitly here
device = unet.device # Get the device the model is on ('cuda')
print("UNet device:", next(unet.parameters()).device)

check_step_zero_only = True

# Prepare models for training
unet.train() # Ensure unet is in train mode (handles dropout etc.)
# VAE and text_encoder should already be frozen and potentially in eval mode if desired
# vae.eval()
# text_encoder.eval()

print(f"Original UNet dtype: {unet.dtype}") # Should show torch.float16
print("Converting UNet to float32 for stability...")
unet.to(torch.float32)
print(f"UNet dtype is now: {unet.dtype}") # Should show torch.float32

print(f"Starting training for {num_train_epochs} epochs...")
print(f"Using learning rate: {optimizer.param_groups[0]['lr']}")

# Training loop
global_step = 0
for epoch in range(num_train_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_train_epochs}")

    for step, batch in enumerate(progress_bar):
        # Ensure batch is not None (due to collate_fn potentially returning None)
        if batch is None:
            print(f"Skipping step {step} in epoch {epoch+1} due to empty batch.")
            continue

        # --- 1. Prepare Batch ---
        # Move pixel values to latent space
        with torch.no_grad():
            # Prepare input in float32
            pixel_values = batch["pixel_values"].to(device, dtype=torch.float32)
            # --- Check Pixel Values ---
            if step == 0 and check_step_zero_only:
                 print(f"Step {step} Check: pixel_values isnan: {torch.isnan(pixel_values).any()}, isinf: {torch.isinf(pixel_values).any()}")
                 if torch.isnan(pixel_values).any() or torch.isinf(pixel_values).any():
                     print("!!! ERROR: NaNs/Infs detected in input pixel_values!")
                     training_failed = True; break # Stop processing

            # <<< Temporarily cast VAE to float32 for the encode operation >>>
            original_vae_dtype = vae.dtype # Store original dtype (likely float16)
            vae.to(torch.float32) # Move VAE parameters to float32

            # Encode using float32 VAE and float32 input
            latents = vae.encode(pixel_values).latent_dist.sample() # Now internal ops use float32

            # <<< Restore VAE to original dtype (optional but recommended for consistency/memory) >>>
            vae.to(original_vae_dtype)

            # Convert resulting latents back to the dtype UNet expects (float16)
            latents = latents.to(dtype=unet.dtype)
            # Apply scaling factor (config value is usually float, result matches latent dtype)
            latents = latents * vae.config.scaling_factor

            print("Latent stats:", latents.min().item(), latents.max().item(), latents.mean().item())

        # Sample noise that we'll add to the latents
        noise = torch.randn_like(latents)
        bsz = latents.shape[0]
        timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (bsz,), device=latents.device).long()
        noisy_latents = scheduler.add_noise(latents, noise, timesteps) # Input latents are now float16

        # Get text embeddings for conditioning
        # Tokenize prompts batch-wise
        prompt_texts = batch["prompts"]
        text_inputs = tokenizer(prompt_texts, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
        text_input_ids = text_inputs.input_ids.to(device)
        with torch.no_grad():
             # Text encoder might be float32 or float16 depending on initial load,
             # ensure its output matches unet expectation if needed.
             encoder_hidden_states = text_encoder(text_input_ids)[0]

        # --- 2. Predict Noise ---
        # Predict the noise residual using the LoRA-adapted UNet
        noisy_latents = noisy_latents.to(unet.dtype)
        encoder_hidden_states = encoder_hidden_states.to(unet.dtype)

        print("encoder_hidden_states:")
        print("  min:", encoder_hidden_states.min().item())
        print("  max:", encoder_hidden_states.max().item())
        print("  mean:", encoder_hidden_states.mean().item())
        print("  has NaN:", torch.isnan(encoder_hidden_states).any().item())
        print("  dtype:", encoder_hidden_states.dtype)

        from diffusers.models.attention_processor import AttnProcessor
        pipe.unet.set_attn_processor({name: AttnProcessor() for name in pipe.unet.attn_processors.keys()})

        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample # UNet runs in float16

        print("NaN check noise_pred and noise:", torch.isnan(noise_pred).any().item(), torch.isnan(noise).any().item())

        # --- 3. Calculate Loss ---
        # MSE Loss between predicted noise and actual noise
        loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

        # --- Check for NaN Loss IMMEDIATELY ---
        if torch.isnan(loss):
            print(f"!!! NaN detected in loss at Epoch {epoch+1}, Step {step}. Stopping. !!!")
            # Consider adding more debug prints here for noise_pred, noise, latents etc.
            # Example: print("NaN Check:", torch.isnan(noise_pred).any(), torch.isnan(noise).any())
            # raise RuntimeError("NaN Loss detected") # Option to stop execution
            # Or just break the inner loop for this example:
            epoch_loss = float('nan') # Mark epoch loss as nan
            break # Stop processing this epoch

        # --- 4. Backpropagation and Optimization ---
        loss = loss / gradient_accumulation_steps
        loss.backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # --- 5. Logging and Updates ---
        global_step += 1
        if not torch.isnan(loss): # Only add if loss is valid
             current_loss = loss.item() * gradient_accumulation_steps
             epoch_loss += current_loss
             progress_bar.set_postfix({"loss": current_loss})
        else:
             progress_bar.set_postfix({"loss": "nan"})

    # --- End of Epoch ---
    # Check if epoch_loss became nan because of the break
    if np.isnan(epoch_loss):
         print(f"Epoch {epoch+1} finished. Average Loss: nan (NaN detected during epoch)")
         # Optionally break the outer loop if one epoch fails entirely
         # break
    elif len(train_dataloader) > 0: # Avoid division by zero if dataloader is empty
        avg_epoch_loss = epoch_loss / len(train_dataloader)
        print(f"Epoch {epoch+1} finished. Average Loss: {avg_epoch_loss:.4f}")
    else:
        print(f"Epoch {epoch+1} finished. DataLoader was empty.")

    # --- Optional: Save LoRA weights periodically ---
    if (epoch + 1) % 10 == 0: # Save every 10 epochs, adjust as needed
        save_path = f"./lora_weights_epoch_{epoch+1}.safetensors"
        try:
            # Use PEFT's save_pretrained for easy saving of adapter weights
            unet.save_pretrained(save_path.replace('.safetensors','')) # Saves to a directory
            # Or save manually if needed:
            # state_dict = unet.state_dict()
            # lora_state_dict = {k: v for k, v in state_dict.items() if 'lora_' in k}
            # torch.save(lora_state_dict, save_path)
            print(f"LoRA weights saved to {save_path.replace('.safetensors','')} directory for epoch {epoch+1}")
        except Exception as e:
             print(f"Error saving weights for epoch {epoch+1}: {e}")


print("\nTraining finished.")

UNet device: cuda:0
Original UNet dtype: torch.float32
Converting UNet to float32 for stability...
UNet dtype is now: torch.float32
Starting training for 3 epochs...
Using learning rate: 0.0001


Epoch 1/3:   0%|          | 0/7 [00:00<?, ?it/s]

Step 0 Check: pixel_values isnan: False, isinf: False
Latent stats: -11.28123950958252 9.135478019714355 0.1552463322877884
encoder_hidden_states:
  min: -28.0625
  max: 33.09375
  mean: -0.11319645494222641
  has NaN: False
  dtype: torch.float32
NaN check noise_pred and noise: True False
!!! NaN detected in loss at Epoch 1, Step 0. Stopping. !!!
Epoch 1 finished. Average Loss: nan (NaN detected during epoch)


Epoch 2/3:   0%|          | 0/7 [00:00<?, ?it/s]

Step 0 Check: pixel_values isnan: False, isinf: False
Latent stats: -6.523621082305908 5.411352634429932 0.08155117928981781
encoder_hidden_states:
  min: -28.0625
  max: 33.09375
  mean: -0.11319645494222641
  has NaN: False
  dtype: torch.float32
NaN check noise_pred and noise: True False
!!! NaN detected in loss at Epoch 2, Step 0. Stopping. !!!
Epoch 2 finished. Average Loss: nan (NaN detected during epoch)


Epoch 3/3:   0%|          | 0/7 [00:00<?, ?it/s]

Step 0 Check: pixel_values isnan: False, isinf: False
Latent stats: -9.313883781433105 7.161795139312744 0.07507482171058655
encoder_hidden_states:
  min: -28.0625
  max: 33.09375
  mean: -0.11319645494222641
  has NaN: False
  dtype: torch.float32
NaN check noise_pred and noise: True False
!!! NaN detected in loss at Epoch 3, Step 0. Stopping. !!!
Epoch 3 finished. Average Loss: nan (NaN detected during epoch)

Training finished.
